https://khlee88.github.io/GRU4Rec_tutorial/

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import time
import os

os.listdir('../paper_practice/data')

['yoochoose-test.dat',
 'yoochoose-buys.dat',
 'yoochoose-clicks.dat',
 'dataset-README.txt']

# Preprocessing

In [2]:
ROOT = '../paper_practice/data/'

checkpoint_dir = './checkpoint'
if not os.path.exists(checkpoint_dir) : os.mkdir(checkpoint_dir)

In [18]:
#load data
colnames = ['Session_ID', 'timestamp', 'Item_ID']
data = pd.read_csv(ROOT + 'yoochoose-clicks.dat', names = colnames, dtype = {'Item_ID' : np.int64},
                  header = 0, usecols = [0,2,1])
valid = pd.read_csv(ROOT + 'yoochoose-test.dat', names = colnames, dtype = {'Item_ID' : np.int64},
                   header = 0, usecols = [0,2,1])

print('data shape : ', data.shape)
print('valid shape : ', valid.shape)

data shape :  (33003943, 3)
valid shape :  (8251790, 3)


In [19]:
data = data[['Session_ID', 'Item_ID', 'timestamp']]
valid = valid[['Session_ID', 'Item_ID', 'timestamp']]

In [5]:
data.head(3)

,Session_ID,Item_ID,timestamp
0,1,214536500,2014-04-07T10:54:09.868Z
1,1,214536506,2014-04-07T10:54:46.998Z
2,1,214577561,2014-04-07T10:57:00.306Z


In [6]:
valid.head(3)

,Session_ID,Item_ID,timestamp
0,5,214530776,2014-04-07T17:20:56.973Z
1,5,214530776,2014-04-07T17:21:19.602Z
2,10,214820942,2014-04-04T07:44:14.590Z


# data preprocessing

In [16]:
def check_data_sort(dt, sample_check = False, sample_size = 10000) :
    if sample_check :
        sess_ids = dt['Session_ID'].unique()
        sample_sess_ids = np.random.choice(sess_ids, sample_size, replace = False)
        dt = dt[np.in1d(dt.Session_ID, sample_sess_ids)]
        #np.in1d(x,y) : x안에 y가 있으면 True
        
    ordered_dt = dt.sort_values(['Session_ID', 'timestamp'])
    return ordered_dt

In [20]:
data = check_data_sort(data, sample_check = True)
valid = check_data_sort(valid)

In [42]:
data['timestamp'] = data['timestamp'].str.replace('T', ' ').apply(lambda x : x.split('.')[0])
valid['timestamp'] = valid['timestamp'].str.replace('T', ' ').apply(lambda x : x.split('.')[0])

from datetime import datetime
import time
data['timestamp'] = data['timestamp'].apply(lambda x : time.mktime(datetime.strptime(x, "%Y-%m-%d %H:%M:%S").timetuple()))
valid['timestamp'] = valid['timestamp'].apply(lambda x : time.mktime(datetime.strptime(x, "%Y-%m-%d %H:%M:%S").timetuple()))

In [52]:
data['timestamp'] = data['timestamp'].apply(lambda x : time.mktime(datetime.strptime(x, "%Y-%m-%d %H:%M:%S").timetuple()))
valid['timestamp'] = valid['timestamp'].apply(lambda x : time.mktime(datetime.strptime(x, "%Y-%m-%d %H:%M:%S").timetuple()))

In [53]:
import gc
gc.collect()

1070

In [25]:
# add item index
itemids = data['Item_ID'].unique()
n_items = len(itemids)
print('the number of item ID : ', n_items)

the number of item ID :  8195


In [62]:
itemidmap = pd.Series(data = np.arange(n_items), index = itemids).to_dict()
data['Item_idx'] = data['Item_ID'].map(itemidmap)
data.head()

,Session_ID,Item_ID,timestamp,Item_idx
365,104,214829741,1.396863e+09,0
366,104,214820842,1.396863e+09,1
367,104,214821279,1.396863e+09,2
368,104,214703790,1.396863e+09,3
938,298,214508942,1.396432e+09,4


In [64]:
#offset sessions
#각 세션의 시작점의 index list

offset_sessions = np.zeros(data['Session_ID'].nunique() + 1, dtype = np.int32)
offset_sessions[1:] = data.groupby('Session_ID').size().cumsum()
offset_sessions[:5]

array([ 0,  4,  6,  9, 13], dtype=int32)

# prepare Model

In [ ]:
layers = 1
rnn_size = 100
batch_size = 50
drop_keep_prob = 0.7

n_epochs = 3
learning_rate = 0.001
decay = 0.96
decay_steps = 1e4
grad_cap = 0
print_step = 1e3

In [69]:
#placeholder & learning rate

#tf.placeholder : 미래의 변수. value는 나중에 할당
X = tf.placeholder(tf.int32, shape = [batch_size], name = 'input')
Y = tf.placeholder(tf.int32, shape = [batch_size], name = 'output')

States = [tf.placeholder(tf.float32, [batch_size, rnn_size], name = 'rnn_state') for _ in range(layers)]
#layers의 값에 따라 placeholder를 list안에 생성하는 코드

global_step = tf.Variable(0, name = 'global_step', trainable = False)
lr = tf.maximum(1e-5, tf.train.exponential_decay(
                learning_rate, #최소 학습시 사용될 learning_rate
                global_step, #현재 학습 횟수
                decay_steps, #총 학습 횟수
                decay, #얼마나 rate가 감소될 것인지(0.96씩 곱해져서 감소한다)
                staircase = True))

In [70]:
#gru weights
with tf.variable_scope('gru_layer', reuse = tf.AUTO_REUSE) :
    initializer = tf.glorot_uniform_initializer()
    embedding = tf.get_variable('embedding', [n_items, rnn_size], initializer=initializer)
    
    softmax_W = tf.get_variable('softmax_w', [n_items, rnn_size], initializer=initializer)
    softmax_b = tf.get_variable('softmax_b', [n_items], initializer=initializer())

TypeError: __call__() missing 1 required positional argument: 'shape'